In [2]:
!pip install faiss-cpu langchain pandas python-docx pypdf openpyxl langchain-openai tiktoken

  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached tiktoken-0.8.0-cp310-cp310-win_amd64.whl.metadata (6.8 kB)
Using cached python_docx-1.1.2-py3-none-any.whl (244 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached tiktoken-0.8.0-cp310-cp310-win_amd64.whl (884 kB)


In [3]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o",
    temperature = 0,
)

In [4]:
base_path = 'd:/data/files/'
all_texts = [] # 추출된 모든 텍스트 데이터를 저장할 저장소를 초기화

In [5]:
import pandas as pd
from langchain.document_loaders import PyPDFLoader
from docx import Document

# Word 파일(.docx)에서 텍스트 추출
def load_docx(path):
    doc = Document(path)
    return "\n".join([p.text for p in doc.paragraphs if p.text.strip() != ''])
# doc.paragraphs를 사용하여 문서 내 모든 문단(paragraphs)을 반복하면서 텍스트를 추출
# strip() != ''을 사용하여 공백 문단 제거
# "\n".join(...)을 사용하여 각 문단을 줄바꿈(\n)을 기준으로 하나의 문자열로 결합

# Excel 파일(.xlsx)에서 텍스트 추출
def load_excel(path):
    xls = pd.ExcelFile(path)
    all_sheets_texts = []
    for sheet_name in xls.sheet_names:
        df = xls.parse(sheet_name)
        all_sheets_texts.append("\n".join(map(str, df.values.ravel())))
    return "\n".join(all_sheets_texts)
# xls.sheet_names: 파일 내 모든 시트 이름을 가져옴
# xls.parse(sheet_name): 각 시트를 Pandas DataFrame으로 변환
# df.values.ravel(): 모든 셀 값을 1차원 배열(numpy.ravel())로 변환
# map(str, df.values.ravel()): 모든 셀 값을 문자열로 변환
# "\n".join(...): 시트 내용을 줄바꿈(\n)을 기준으로 결합

# PDF 파일에서 텍스트 추출
def load_pdf(path):
    try:
        doc = PyPDFLoader(path).load()
        return "\n".join([page.page_content for page in doc])
    except Exception as e: # PDF 파일이 손상되었거나 읽을 수 없을 경우 오류 메시지를 출력
        print(f"Error processing PDF {path}. Error: {e}")
        return ""
# [page.page_content for page in doc]: 모든 페이지에서 텍스트(page_content) 추출
# "\n".join(...): 각 페이지의 텍스트를 줄바꿈(\n)을 기준으로 결합

In [6]:
# base_path 폴더 내 모든 파일과 하위 폴더를 탐색
for subdir, dirs, files in os.walk(base_path): 
    # 파일 확장자 확인 및 적절한 함수 호출
    for file in files:
        filename = os.path.join(subdir, file) # 파일의 전체 경로 생성 (예, d:/data/files/report.pdf)
        extension = os.path.splitext(filename)[1].lower() # 파일 확장자 추출 및 소문자로 변환

        # 파일 타입에 따라 적절한 함수 호출
        text_content = ""
        try:
            if extension == ".pdf":
                text_content = load_pdf(filename)
            elif extension == ".docx":
                text_content = load_docx(filename)
            elif extension in [".xls", ".xlsx"]:
                text_content = load_excel(filename)

        # 예외 처리
        except Exception as e:
            print(f"Error processing file {filename}. Error: {e}")

        # 텍스트가 정상적으로 추출되었을 경우(if text_content:), all_texts에 추가
        if text_content:
            all_texts.append(text_content)

In [7]:
from langchain import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 모든 문서를 하나의 문자열로 합침
combined_texts = "\n".join(all_texts)

# RecursiveCharacterTextSplitter를 사용하여 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_text(combined_texts) # 하나의 긴 문서를 여러 개의 청크로 분할

# OpenAI 임베딩 모델 로드
embeddings = OpenAIEmbeddings()

# 변환된 벡터를 FAISS 벡터 데이터베이스에 저장
vectorstore = FAISS.from_texts(texts, embeddings) 
vectorstore.save_local('d:/data/db_faiss_combined') # 추후 재사용을 위해 FAISS 벡터 데이터를 로컬 디렉토리에 저장

In [8]:
# FAISS 데이터베이스 로드 (FAISS.load_local)
retriever = FAISS.load_local(
    "d:/data/db_faiss_combined",  # 저장된 FAISS 데이터베이스 경로
    OpenAIEmbeddings(),  
    allow_dangerous_deserialization=True  
).as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 8}
) # as_retriever()를 사용하여 문서 검색 가능하도록 변환

In [9]:
from langchain.chains import RetrievalQA

# RetrievalQA.from_chain_type()을 사용하여 QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  
    retriever=retriever, # FAISS 데이터베이스에서 유사한 문서를 검색
)

# 엑셀 기반의 질문
query = "마음 챙김의 리뷰는?"
response = qa_chain.invoke(query)
print(response)

{'query': '마음 챙김의 리뷰는?', 'result': '마음 챙김에 대한 리뷰는 다음과 같습니다:\n\n1. "페이지를 넘길 수밖에 없었어요!" - 평점 1.9\n2. "생각보다 별로였어요." - 평점 1.2'}


In [10]:
# 워드 기반의 질문
query = "집중력 향상을 위해 좋은 활동은?"
response = qa_chain.invoke(query)
print(response)

{'query': '집중력 향상을 위해 좋은 활동은?', 'result': '집중력 향상을 위해 좋은 활동으로는 퍼즐과 보드게임이 있습니다. 퍼즐이나 보드게임은 논리적 사고와 집중력을 필요로 하므로 두뇌 운동에 매우 효과적입니다. 크로스워드 퍼즐, 스도쿠 풀기, 체스, 바둑, 장기 같은 전략적인 보드게임을 추천합니다.'}
